In [16]:
import numpy as np
import math
import os
import scipy.io as spio
import sys
# np.set_printoptions(threshold=sys.maxsize)

In [17]:
inband = 4
bandnum = 10
threshold = -145
slope = 12

In [18]:
midfre = [16, 20, 26, 36, 48, 60, 80, 101, 256, 353]
second_range = 512
icfilter = np.zeros((bandnum, second_range),dtype=float)

In [19]:
def write_speech_to_mat(source, destination):
    pass

In [20]:
def load_file(file_name):
    pass

In [21]:
def load_matrix_weight(mat_file):
    pass

In [22]:
def genICFilterMatrix(bandnum, y_length, mid_freq_matrix):
    ret_icfilter = np.zeros((bandnum, y_length),dtype=float)
    for i in range(bandnum):
        if i == 9:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-360*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
        elif i == 10:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-360*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
        else:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-20*slope*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
    
    return ret_icfilter

In [23]:
#test genICFilterMatrix
# print("midfre is {}".format(midfre))
# icfarray = genICFilterMatrix(10,512,midfre)
# print(icfarray[1])
# print("shape of row 1 is {}".format(icfarray[1].shape))

In [24]:

# for i in range(bandnum):
    
#     if i == 9:
#         for j in range(second_range):
#             icfilter[i][j]=10**((-360*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(midfre[i]-1))))/20)
#     elif i == 10:
#         for j in range(second_range):
#             icfilter[i][j]=10**((-360*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(midfre[i]-1))))/20)
#     else:
#         for j in range(second_range):
#             icfilter[i][j]=10**((-360*slope*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(midfre[i]-1))))/20)            

In [25]:
# count_answer_1 = 0
# count_answer_2 = 0
# count_test_1 = 0
# count_test_2 = 0

In [26]:
# y_answer = np.empty([1,1])
# y_test = np.empty([1,1])

In [27]:
# local_power = np.empty([0,0])
# local_power_normalize = np.empty([1,1])
# x_all = np.empty([1,1])
# x_normalize_all = np.empty([1,1])

In [28]:
#load mat file
weights = spio.loadmat('../weight/8+2band(25ms)/20200110/weight_512x32_20200110_15786815140978317.mat')
w1 = weights["w1"]
w2 = weights["w2"]
w3 = weights["w3"]
b1 = weights["b1"]
b2 = weights["b2"]
b3 = weights["b3"]
s = spio.loadmat('../data/noise_mat/Speakers_Train/bear.dat.mat')
target = 0 # 0: mean we load the noise data; 1: load the human speech
# print("s length is {}".format(len(s)))


In [29]:
def sigmoid(X):
    return 1/(1+np.exp(-X))

def relu(X):
    return np.maximum(0,X)

def softmax(X):
    expo = np.exp(X)
    expo_sum = np.sum(np.exp(X))
    return expo/expo_sum

In [30]:

# with open("icfileter_dump.txt","w") as fw2:
#         fw2.write(str(icfilterary_final))

# print(icfilterary)
# print("icfilterary_final shape is {}".format(icfilterary_final.shape))

# print("length of icfilterary is {}".format(len(icfilterary)))
# print(400*(np.floor(len(s['s']))))

# print("s shape is {}".format(s['s'].shape))
# icfilterary = genICFilterMatrix(10,514,midfre)
count_answer_1 = 0
count_answer_0 = 0
count_test_1 = 0
count_test_0 = 0

y_answer = np.empty([0,0])
y_test = np.empty([0,0])

local_power = np.empty([0,0])
local_power_normalize = np.empty([0,0])
x_all = np.empty([0,0])
x_normalize_all = np.empty([0,0])
threshold = -145
#***************************
y_len = 513
icfilterary = genICFilterMatrix(10,y_len,midfre)[:,1:]#icfilterary[:,1:]
# print(icfilterary.shape)
s_len = int(400*(np.floor(len(s['s']))))
sample_num = inband * bandnum
test_counter = 0
print("w1 shape is {}".format(w1.shape))
print("w2 shape is {}".format(w2.shape))
print("w3 shape is {}".format(w3.shape))
for i in range(1600,s_len,400):
#     if test_counter > 0:
#         break
    test_counter += 1
    s1 = s['s'][i-1600:i-1200]#0-399
    s2 = s['s'][i-1200:i-800] #400-799
    s3 = s['s'][i-800:i-400]  #800-1199
    s4 = s['s'][i-400:i]      #1200-1599
    
    s_fft_1 = np.fft.fft(s1,n=1024,axis=0) 
    s_fft_2 = np.fft.fft(s2,n=1024,axis=0) 
    s_fft_3 = np.fft.fft(s3,n=1024,axis=0) 
    s_fft_4 = np.fft.fft(s4,n=1024,axis=0) 
    
    s_fft_1_513 = np.split(s_fft_1,[0,y_len-1],axis=0)[1]
    s_fft_2_513 = np.split(s_fft_2,[0,y_len-1],axis=0)[1]
    s_fft_3_513 = np.split(s_fft_3,[0,y_len-1],axis=0)[1]
    s_fft_4_513 = np.split(s_fft_4,[0,y_len-1],axis=0)[1]
    
    # there is one error element in every array in position 9 of each array
    x1 = np.log(np.matmul(icfilterary,(np.abs(s_fft_1_513)**2)) + 0.0001)
    x2 = np.log(np.matmul(icfilterary,(np.abs(s_fft_2_513)**2)) + 0.0001)
    x3 = np.log(np.matmul(icfilterary,(np.abs(s_fft_3_513)**2)) + 0.0001)
    x4 = np.log(np.matmul(icfilterary,(np.abs(s_fft_4_513)**2)) + 0.0001)
    
    x= np.array([x1,x2,x3,x4]).reshape(1,sample_num)
#     print(x)
    x_40 = x[0:40]
    max_ele = np.amax(x_40,axis=1)
    min_ele = np.amin(x_40,axis=1)
    sum_of_x = np.sum(x_40)
    
    
#     print(x_all.reshape(1,len(x_all)).shape)
#     print(x_all)
    
#     print(x1)
#     print("********************")
#     print(x2)
#     print("********************")
#     print(x3)
#     print("********************")
#     print(x4)

#     x_all = np.append(x_all,x_40)
#     local_power = np.append(local_power, sum_of_x)

    # normalize
    x_normalize = (x_40-min_ele)/(max_ele-min_ele+0.0001)
#     print("x_normalize shape is {}".format(x_normalize.shape))
#     print(x_normalize[0])
#     x_normalize_all = np.append(x_normalize_all, x_normalize)
#     local_power_normalize = np.append(local_power_normalize,np.sum(x_normalize[0]))
                                 
    #counting the test
    if sum_of_x > threshold:
        #nn.Softmax(nn.ReLU(nn.ReLU(x_normalize*w1+b1)*w2+b2)*w3+b3)
        #performing the model weight mulplications
        answer = softmax(np.matmul(relu(np.matmul(relu(np.matmul(x_normalize,w1)+b1),w2)+b2),w3)+b3)
        print(answer[0,0])
        print(answer[0,1])
        print("************************\n")
        if answer[0,0] > answer[0,1]:
            y_test = np.append(y_test, 0)
            count_test_0 += 1
        else:
            y_test = np.append(y_test, 1)
            count_test_1 += 1
    else:
        y_test = np.append(y_test, 0)
        count_test_0 += 1
        
    #counting our answer
    if target == 1:
        if sum_of_x > threshold:
            y_answer = np.append(y_answer,1)
            count_answer_1 += 1
        else:
            y_answer = np.append(y_answer,0)
            count_answer_0 += 1
    else:
        y_answer = np.append(y_answer,0)
        count_answer_0 += 1
        
    
print("x_normalize is {}".format(x_normalize))
print("y_test is {}".format(y_test))
print("y_answer is {}".format(y_answer))

/home/rickliao/.conda/envs/pytorchenv/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in log10
  if sys.path[0] == '':
/home/rickliao/.conda/envs/pytorchenv/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log10
  


w1 shape is (40, 512)
w2 shape is (512, 32)
w3 shape is (32, 2)
0.9964157590526413
0.0035842409473586237
************************

0.9996090895792641
0.00039091042073584857
************************

0.9997095746942202
0.0002904253057797433
************************

0.9984545949211795
0.0015454050788203967
************************

0.9997588899860718
0.00024111001392820977
************************

0.9999024450291915
9.755497080852882e-05
************************

0.9999964016299767
3.598370023268617e-06
************************

0.9999999995974361
4.025639392793953e-10
************************

0.9999987393786394
1.2606213606836532e-06
************************

0.9999978980171771
2.1019828230035774e-06
************************

0.999999836960562
1.630394379936583e-07
************************

0.9999904026383268
9.597361673169739e-06
************************

0.9999969906020654
3.0093979346744604e-06
************************

0.9999998659109486
1.3408905142260615e-07
*******************

x_normalize is [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
y_test is [0. 0. 0. ... 0. 0. 0.]
y_answer is [0. 0. 0. ... 0. 0. 0.]


In [33]:
acc = 0
LED = 0
wrong = np.empty([0,0])

y_answer_len = len(y_answer)
y_test_len = len(y_test)
if target == 1: #if our test data is human voice
    for c in range(y_answer_len):
        acc += abs(y_answer[c]-y_test[c])
    
    acc = (y_answer_len-acc)/y_answer_len
else:
    acc = count_test_0/(count_test_0+count_test_1)

print("accuracy is {}".format(acc))

accuracy is 0.999737493437336


In [ ]:
# this cell is for aduiowrite of answer and test

In [103]:
tmp_ary = np.array([1,1])
for i in range(3):
    tmp_ary2 = np.random.randint(low = 1, high = 99, size = 513)
    tmp_ary = np.append(tmp_ary,tmp_ary2)
tmp_ary = tmp_ary.reshape(len(tmp_ary),1)
print(tmp_ary.shape)

(1541, 1)
